<a href="https://colab.research.google.com/github/JakeOh/202505_BD50/blob/main/lab_da/ml21_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM(Large Language Model, 거대 언어 모델)

*   시퀀스-시퀀스 작업(Sequence-to-Sequence)
    *   시퀀스 데이터를 입력받아서 시퀀스 데이터를 출력하는 작업
    *   자연어 처리(NLP, Natural Language Processing) 분야에서 요약, 번역 등의 작업 해당
    *   두 개의 (순환) 신경망을 연결한 인코더-디코더(encoder-decoder) 구조가 널리 사용
*   어텐션 메커니즘(Attention mechanism)
    *   인코더-디코더 구조에서 사용된 순환 신경망의 성능을 향상시키기 위해서 고안
    *   기존에는 인코더의 마지막 타임스텝에서 출력한 은닉 상태만을 사용해서 디코더가 새로운 텍스트를 생성
    *   어텐션 메커니즘은 모든 타임스텝에서 인코더가 출력한 은닉 상태를 디코더가 참조할 수 있도록 고안
    *   디코더가 새로운 토큰을 생성할 때 인코더가 처리한 토큰들 중에서 어떤 토큰에 주의(attention)를 기울일 지를 결정. (입력 토큰들마다 디코더가 중요도를 다르게 부여)
*   트랜스포머 모델(Transformer model)
    *   어텐션 메커니즘을 기반으로 하여 인코더-디코더 구조에서 순환층을 제거
    *   인코더에서 한 번에 하나의 토크씩 처리하지 않고 입력 테스트 전체를 한 번에 처리
    *   핵심 구성 요소
        *   멀티 헤드 어텐션(multi-head attention)
        *   층 정규화(layer normalization)
        *   잔차 연결(residual connection)
        *   피드 포워드 네트워크(feed-forward network)

<img src="https://pbs.twimg.com/media/GJg3QtMXwAAvUT4?format=jpg&name=large"
    alt="LLM 가계도" />

*   Encoder 기반 모델
    *   텍스트 (긍정/부정) 분류
    *   개체명 인식 - 텍스트에서 사람 이름, 지역 이름, 회사 이름 등의 고유 명사를 식별.
    *   BERT, RoBERTa, ...
*   Encoder-Decoder 기반 모델
    *   문서 요약, 번역, 질문-답변(질문과 문맥 텍스트가 주어졌을 때 문맥 안에서 답을 찾아서 생성)
    *   T5, BART, ...
*   Decoder 기반 모델
    *   텍스트 생성 - 챗봇, 질문 답변, 요약, 번역
    *   디코더:
        *   이전까지 생성한 텍스트를 입력받아 다음 토큰을 예측하는 방식
        *   인코더로부터의 입력이 없으면 디코더는 아무것도 생성할 수 없음.
        *   이전에 생성한 텍스트인 것처럼 어떤 텍스트를 입력해 주면 인코더 도움 없이 다음 토큰을 예측.
        *   프롬프트(prompt): 이전에 생성한 텍스트인 것처럼 전달하는 초기 텍스트.
    *   GPT-4, GPT-5, LLaMA, ...
    *   현재 가장 활발히 연구되고 있는 LLM 분야.

# HuggingFace에서 KoBART 모델 사용하기

In [1]:
import transformers  # HuggingFace를 사용하기 위한 패키지

In [2]:
# HuggingFace에서 오픈 소스로 공개된 모델을 다운로드해서 로컬에서 실행할 수 있도록 해줌.
pipe = transformers.pipeline(task="summarization",  # 문서 요약 작업
                             model="sshleifer/distilbart-cnn-12-6",  # 모델
                             device=0)  # device=0: GPU 사용

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0
